In [2]:
import folium
import geopandas as gdp
import pandas as pd

geoPP = gdp.read_file("territoiresPP_Qc.json")

geoPP.head()

,id,Slug,Name,color,description,geometry
0,903,huron-wendat,Wendake-Nionwentsïo,#E6EABB,https://native-land.ca/maps/territories/huron-...,"MULTIPOLYGON (((-73.06725 46.41254, -72.89482 ..."
1,489,naskapi,ᒋᑦ ᐊᔅᒋᓄᐤ St’aschinuw (Naskapi),#ACF78E,https://native-land.ca/maps/territories/naskapi,"MULTIPOLYGON (((-64.44209 57.81602, -65.01576 ..."
2,824,cree-nehiyawak-of-eeyou-istchee,Eeyou Istchee,#FDB845,https://native-land.ca/maps/territories/cree-n...,"MULTIPOLYGON (((-79.8706 54.73731, -79.80469 5..."
3,145,mohawk,Kanienʼkehá꞉ka (Mohawk),#796569,https://native-land.ca/maps/territories/mohawk,"MULTIPOLYGON (((-73.34009 44.70184, -73.3289 4..."
4,675,wabanaki-confederacy,Wabanaki (Dawnland Confederacy),#CCBEB9,https://native-land.ca/maps/territories/wabana...,"MULTIPOLYGON (((-69.76666 43.40904, -68.61014 ..."


In [3]:
geoPP = geoPP[["Slug", "geometry"]]
geoPP.head()

,Slug,geometry
0,huron-wendat,"MULTIPOLYGON (((-73.06725 46.41254, -72.89482 ..."
1,naskapi,"MULTIPOLYGON (((-64.44209 57.81602, -65.01576 ..."
2,cree-nehiyawak-of-eeyou-istchee,"MULTIPOLYGON (((-79.8706 54.73731, -79.80469 5..."
3,mohawk,"MULTIPOLYGON (((-73.34009 44.70184, -73.3289 4..."
4,wabanaki-confederacy,"MULTIPOLYGON (((-69.76666 43.40904, -68.61014 ..."


In [4]:
df = pd.read_csv("df_PP.csv")
df.head()

,id,Slug,Name,Pop2022,bonjout_salut,merci
0,903,huron-wendat,Wendake-Nionwentsïo,4771,Kwe ! Ndio !,Tiawenhk
1,489,naskapi,ᒋᑦ ᐊᔅᒋᓄᐤ St’aschinuw (Naskapi),1592,Waachiyaa,Chiniskumitin
2,824,cree-nehiyawak-of-eeyou-istchee,Eeyou Istchee,20902,waachiyaah,miikwech
3,145,mohawk,Kanienʼkehá꞉ka (Mohawk),20457,Kwe kwe / Shé:kon,Niáwen
4,675,wabanaki-confederacy,Wabanaki (Dawnland Confederacy),3565,Kwaï,Wlini


In [5]:
PP_gdf = geoPP.merge(df, on="Slug")
PP_gdf.head()

,Slug,geometry,id,Name,Pop2022,bonjout_salut,merci
0,huron-wendat,"MULTIPOLYGON (((-73.06725 46.41254, -72.89482 ...",903,Wendake-Nionwentsïo,4771,Kwe ! Ndio !,Tiawenhk
1,naskapi,"MULTIPOLYGON (((-64.44209 57.81602, -65.01576 ...",489,ᒋᑦ ᐊᔅᒋᓄᐤ St’aschinuw (Naskapi),1592,Waachiyaa,Chiniskumitin
2,cree-nehiyawak-of-eeyou-istchee,"MULTIPOLYGON (((-79.8706 54.73731, -79.80469 5...",824,Eeyou Istchee,20902,waachiyaah,miikwech
3,mohawk,"MULTIPOLYGON (((-73.34009 44.70184, -73.3289 4...",145,Kanienʼkehá꞉ka (Mohawk),20457,Kwe kwe / Shé:kon,Niáwen
4,wabanaki-confederacy,"MULTIPOLYGON (((-69.76666 43.40904, -68.61014 ...",675,Wabanaki (Dawnland Confederacy),3565,Kwaï,Wlini


In [15]:
import branca.colormap as cm

m = folium.Map(location=[53.9073, -68.8623], tiles= None, zoom_start=5)
folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(m)

folium.Choropleth(geo_data=PP_gdf,
             name="Les Premiers Peuples du Kepek",         
             data=PP_gdf,
             columns=["Slug","Pop2022"], 
             key_on="feature.properties.Slug", 
             fill_color="viridis",
             fill_opacity=0.6, 
             line_opacity=0.1,
          #   threshold_scale=scale,     
             legend_name="Populaton des Premiers Peuples des territoires du Kepek", 
             smooth_factor=0,     
             highlight=True,
                 ).add_to(m)

# ajouter un tooltip
style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}

infoRegion = folium.features.GeoJson(
    PP_gdf,
    style_function=style_function, 
    control=False,
    tooltip=folium.features.GeoJsonTooltip(
        fields=["Name", "Pop2022", "bonjout_salut","merci"],
        aliases=["Nom du Premier Peuple:","Population (nombre de personnes) :","Salut bonjour! :","Merci ! :"],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)
m.add_child(infoRegion)
m.keep_in_front(infoRegion)
folium.LayerControl().add_to(m)
m.save("index.html")
m